In [5]:
import pandas as pd

In [6]:
nationalities = [
    "australian",
    "french",
    "indian",
    "korean",
    "canadian",
    "mexican"
]

floors = [
    "6th",
    "5th",
    "4th",
    "3rd",
    "2nd",
    "1st"
]

pets = [
    "spider",
    "fish",
    "parrot",
    "horse",
    "dog",
    "cat"
]

music = [
    "bossa nova",
    "jazz",
    "salsa",
    "classical",
    "rock",
    "opera"
]

In [7]:
feat_table = pd.concat([
    pd.Series(floors, name="floors"),
    pd.Series(nationalities, name="nationalities"),
    pd.Series(pets, name="pets"),
    pd.Series(music, name="music")
], axis=1)
feat_table

,floors,nationalities,pets,music
0,6th,australian,spider,bossa nova
1,5th,french,fish,jazz
2,4th,indian,parrot,salsa
3,3rd,korean,horse,classical
4,2nd,canadian,dog,rock
5,1st,mexican,cat,opera


In [8]:
class Node:

    def __init__(self, name, type, value=None, parent=None, metadata=None):
        self.name = name
        self.type = type
        self.value = value
        self.parent = parent
        self.metadata = metadata

In [60]:
class EinsteinCSP():

    def __init__(self):
        self.nodes = {}
        self.edges = {}

    def add_node(self, node: Node):
        self.nodes[node.name] = node
        self.edges[node.name] = {}

    def add_edge(self, node_a, node_b, constraint, dir=False):
        if node_a.name not in self.nodes.keys(): self.add_node(node_a)
        if node_b.name not in self.nodes.keys(): self.add_node(node_b)
        self.edges[node_a.name][node_b.name] = constraint
        if not dir: self.edges[node_b.name][node_a.name] = constraint

    @staticmethod
    def __different_value(x, y):
        if x.value is None or y.value is None: return False
        return x.value.name != y.value.name
    
    @staticmethod
    def __same_parent(x, y):
        if x.parent is None or y.parent is None: return False
        return x.parent.metadata["row"] == y.parent.metadata["row"]

    @staticmethod
    def __build_cell_name(feature, i):
        return f"{feature}_{i}"
    
    def load_from_feat_table(self, feat_table):
        
        # Load nodes
        for i, row in feat_table.iterrows():
            for feature, value in row.to_dict().items():
                cell_name = self.__build_cell_name(feature, i)
                self.add_node(Node(cell_name, "cell", metadata={"feature":feature, "row":i}))
                self.add_node(Node(value, "value", metadata={"feature":feature, "row":i}))
                self.nodes[cell_name].value = self.nodes[value]
                self.nodes[value].parent = self.nodes[cell_name]

        # Create constraints
        for node_a in self.nodes.values():
            for node_b in self.nodes.values():
                if node_a.name == node_b.name: continue
                if(node_a.type == "cell" and node_b.type == "cell" and
                   node_a.metadata["feature"] == node_b.metadata["feature"]):
                    self.add_edge(node_a, node_b, self.__different_value)
                if(node_a.type == "value" and node_b.type == "value" and
                   node_a.metadata["row"] == node_b.metadata["row"]):
                    self.add_edge(node_a, node_b, self.__same_parent)
    
    def is_consistent(self):
        for node_a, relations in self.edges.items():
            for node_b, constraint in relations.items():
                if not constraint(self.nodes[node_a], self.nodes[node_b]):
                    print(f"{node_a} and {node_b} breaks '{constraint.__name__}' constraint")
                    return False
        return True



In [64]:
csp = EinsteinCSP()
csp.load_from_feat_table(feat_table)
csp.is_consistent()

True

In [66]:
csp.edges

{'floors_0': {'floors_1': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'floors_2': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'floors_3': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'floors_4': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'floors_5': <function __main__.EinsteinCSP.__different_value(x, y)>},
 '6th': {'australian': <function __main__.EinsteinCSP.__same_parent(x, y)>,
  'spider': <function __main__.EinsteinCSP.__same_parent(x, y)>,
  'bossa nova': <function __main__.EinsteinCSP.__same_parent(x, y)>},
 'nationalities_0': {'nationalities_1': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'nationalities_2': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'nationalities_3': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'nationalities_4': <function __main__.EinsteinCSP.__different_value(x, y)>,
  'nationalities_5': <function __main__.EinsteinCSP.__different_value(x, y)>},
 'australia